<a href="https://colab.research.google.com/github/ariG23498/examples/blob/ariG23498%2Fsample-log/colabs/wandb-log/Performant_Sampled_Logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />

<div><img /></div>

<img src="https://i.imgur.com/uEtWSEb.png" width="650" alt="Weights & Biases" />

<div><img /></div>

# Batching Logs for Higher Performance

Sometimes, a large number of small pieces of information need to be logged to Weights & Biases:
- loss on every batch 
- reward on each step of each episode

This can lead to substantial slowdown if the logging calls are much slower than the training iterations -- training will finish, but `wandb` will still be catching up.

In this notebook we will show you how to improve the performance of `wandb.log()` in this case by sampling logs.

# Setup

In [1]:
%%capture
# install the package
! pip install wandb -qq

In [ ]:
# login with the credentials
! wandb login

In [3]:
import wandb
import random

timing_header = "=" * 5 + " Timing Results " + "=" * 5

# Simple, but Slow Method

> In this section we log metrics for each and every iteration.

<font color="red">Warning!</font> This cell takes a few minutes to run.

If you just want to see the fast version, skip it and run the next code cell.

In [ ]:
%%time

total_steps = 100_000

with wandb.init(project="batched-log", name="before") as run:

    for ii in range(total_steps):
        run.log({"metric": random.randint(1, 100)})

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)

# More Performant Method with Batching

> In the code below we will sample iterations and log the mean of the metrics. With this approach we will be boosting the logging process and not loose information too.

In [ ]:
%%time

total_steps = 100_000
log_every = 500
metric = []

with wandb.init(project="batched-log", name="after") as run:

    for ii in range(total_steps):
        metric.append(random.randint(1, 100))

        if (ii + 1) % log_every == 0:
            mean_metric = sum(metric)//log_every
            run.log({"mean_metric":mean_metric})
            metric = []

    print("Run Finished!")

print("Logging Finished!")
print("\n" + timing_header)